In [ ]:
import pandas as pd
import numpy as np
import os
import re
import string

In [ ]:
file_list = os.listdir("../data/ground_truth")
#create and save error terms in error_list
error_df = pd.DataFrame()
tess_error_list = list()
ground_error_list = list()

for file in file_list:
    #read ground_truth lines
    ground_truth_file_path = "../data/ground_truth_trimmed/"+str(file)
    ground_truth = open(ground_truth_file_path, "r",encoding="utf8")
    ground_truth_lines = ground_truth.read().split('\n')
    
    #read tesseract lines
    tesseract_file_path = "../data/tesseract/"+str(file)
    tesseract = open(tesseract_file_path, "r",encoding="utf8") 
    tesseract_lines = tesseract.read().split('\n')
    
    #print(tesseract_lines)
    #print(ground_truth_lines)
    
    #define function: get words split by space
    def get_words(lines):
        lines_list = list()
        for element in lines:
            #element  = element.split(" ")         
            element = [ word for word in element.split(" ") if not word.isdigit() and word not in string.punctuation]
            lines_list.append(element)
        return lines_list
    
    #create documents for tesseract and ground truth
    tesseract_doc = get_words(tesseract_lines)
    ground_truth_doc = get_words(ground_truth_lines)
    
    tess_len = len(tesseract_doc)
    ground_len = len(ground_truth_doc)
    
    ###############################################
    #######      Check for Mismatch          ######
    ###############################################
    
    ##if tess_len != ground_len:
    ##    print(file)
    
    #####After Trimming 13 mismatched files, no mismatch in number of lines detected
    
    
    
    #create and save error terms in error_list
    #error_df = pd.DataFrame()
    #tess_error_list = list()
    #ground_error_list = list()
    for i in range(len(tesseract_doc)):
        if len(tesseract_doc[i]) == len(ground_truth_doc[i]):
            tess = tesseract_doc[i]
            ground = ground_truth_doc[i]
            for j in range(len(tess)):
                #if tess[j] != ground[j]:
                    tess_error_list.append(tess[j])
                    ground_error_list.append(ground[j])
                    
error_df['Tesseract'] = tess_error_list
error_df['Ground_Truth'] = ground_error_list

In [ ]:
###############################################
#######      Error Detection Part         #####
###############################################
import string
import itertools
import re

def error_detect(word):
    #rule_1
    rule_1 = len(list(word))>20
    
    #rule_2
    num_punct = len([char for char in word if char in string.punctuation])
    num_alpha = len([char for char in word if char.isalpha()])
    rule_2 = num_punct>num_alpha
    
    #rule_3 
    rule_3 = len(set([char for char in word[1:-1] if char in string.punctuation]))>=2
    
    #rule_4
    rule_4 = max([len(list(v)) for k,v in itertools.groupby(word)])>=3
    
    #rule_5
    num_upper = len([char for char in word if char.isupper()])
    num_lower = len([char for char in word if char.islower()])
    rule_5 = num_upper > num_lower and num_upper < len(word)
    
    #rule_6
    if word.isalpha():
        vowels = 'aeiouAEIOU'
        num_vowels = len([char for char in word if char in vowels])
        num_consonants = len(word) - num_vowels       
        rule_6 = num_vowels > 8*num_consonants or num_consonants > 8*num_vowels
    else : rule_6 = False
    
    #rule_7
    try:
        vowels_len = max([len(w) for w in re.findall(r'[aeiou]+',word,re.IGNORECASE)])
    except ValueError:
        vowels_len = 0 
    try:
        conson_len = max([len(w) for w in re.findall(r'[^aeiou]+',word,re.IGNORECASE)])
    except ValueError:
        conson_len = 0      
    rule_7 = vowels_len>=4 or conson_len >=5
    
    #rule_8
    rule_8 = word[0].islower() and word[-1].islower() and word[1:-1].isupper()

    return rule_1 or rule_2 or rule_3 or rule_4 or rule_5 or rule_6 or rule_7 or rule_8

#######Detect Error #######
error_df.insert(loc =2, column="IS_error", value= error_df.Tesseract.map(error_detect))

In [ ]:
def remove_punc_digit(string):
    string = re.sub(r'[^\w\s]','',string)
    string = re.sub(r'[^\D]','',string)
    return string

error_df['Tesseract'] = error_df['Tesseract'].apply(lambda x: remove_punc_digit(x))
error_df['Ground_Truth'] = error_df['Ground_Truth'].apply(lambda x: remove_punc_digit(x))

In [ ]:
#for index in range(len(error_df)):
    #if error_df['Tesseract'][index].isdigit():
        #error_df['Tesseract'][index] = None
    #if error_df['Ground_Truth'][index].isdigit():
        #error_df['Ground_Truth'][index] = None

In [ ]:
error_df = error_df[error_df.IS_error==True]
error_df = error_df.drop(columns="IS_error")
error_df.Tesseract.replace('', np.nan, inplace=True)
error_df.Ground_Truth.replace('', np.nan, inplace=True)
error_df = error_df.dropna()
error_df = error_df.reset_index(drop = True)

In [ ]:
error_df.to_csv("../data/Error_df_rules_based.csv")

In [ ]:
error_df